# Headlines

In [2]:
import spacy

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from src.pipeline_config import get_default_args, get_checkpoint

In [5]:
from transformers import BertTokenizer
from src.bertsum import AbsSummarizer
from razdel import sentenize, tokenize
import torch
from src.generator import build_predictor

C:\Users\Orbis\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Orbis\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Orbis\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Orbis\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passi

In [6]:
args=get_default_args()

In [7]:
args.min_length=3

In [8]:
args.beam_size=10

In [9]:
class batch:
    def __init__(self,src,segs,mask):
        self.src=src
        self.segs=segs
        self.mask_src=mask
        self.batch_size=src.shape[0]

def mark_segments(ids,sep_id):
    seg_id=0
    res=[0]*len(ids)
    for i, tid in enumerate(ids):
        res[i]=seg_id
        if tid==sep_id:
            seg_id=(seg_id+1)%2
    return res

def prepare_text_batch(texts, tokenizer, sep_token='[SEP]', cls_token='[CLS]',max_len=512):
    example=tokenizer.batch_encode_plus(texts,padding=True, add_special_tokens=False, return_tensors='pt', truncation=True, max_length=512)
    ids=example['input_ids']
    mask=~(ids==0)
    segs=mark_segments_tensor(ids, sid=tokenizer.vocab[sep_token])
    return ids, segs, mask
    
def prepare_text(text, tokenizer, sep_token='[SEP]', cls_token='[CLS]',max_len=512):
    tokens=[t for s in sentenize(text) for t in tokenizer.tokenize(cls_token+s.text+sep_token)]
    ids=tokenizer.convert_tokens_to_ids(tokens)[:max_len]
    segs=mark_segments(ids,tokenizer.vocab[sep_token])
    ids=torch.tensor(ids)
    segs=torch.tensor(segs)
    mask=~(ids==0)
    return ids.view(1,-1), segs.view(1,-1), mask.view(1,-1)

def decode_prediction(tokenizer,prediction):
    return " ".join([tokenizer.ids_to_tokens[i] for i in prediction]).replace(" ##","")

def nested_shape(l):
    if isinstance(l,list):
        return [len(l)] + nested_shape(l[0])
    else:
        return []

def insert_special_toks(text, sep_token='[SEP]', cls_token='[CLS]'):
    return " ".join([cls_token+s.text+sep_token for s in sentenize(text)])

def mark_segments_tensor(example,sid):
    zz=torch.zeros(example.shape, dtype=torch.long)
    segs=torch.where(example==sid)
    prev=None
    flag=0
    for x,y in zip(*segs):
        if prev and prev[0]!=x:
            flag=0
        elif prev:
            zz[x,prev[1]:y]=flag
            flag=(flag+1)%2
        prev=(x,y)
    return zz

In [10]:
tokenizer = BertTokenizer.from_pretrained(args.ru, do_lower_case=False, cache_dir=args.temp_dir)

In [11]:
data_path="../Parser/Crawler/NewsParsing/Testing/headline_dataset/test.jsonl"

In [12]:
import json

def read_data(path):
    res=[]
    with open(path, encoding="utf-8") as f:
        for l in f:
            res.append(json.loads(l))
    return res

In [13]:
data=read_data(data_path)

In [14]:
data[3]

{'target': 'Несколько районов Молдавии серьезно пострадали из-за стихии',
 'source': 'В результате ливневых дождей и крупного града более 50 населенных пунктов Молдавии остаются без электричества, сообщили "Интерфаксу" в бюро правительства по связям с прессой. "Были разрушены крыши более тысячи домов, несколько десятков населенных пунктов отрезаны от электричества. В настоящее время ведутся восстановительные работы, размер ущерба подсчитывается. Без света остаются еще 52 населенных пункта", - сказал председатель Службы гражданской защиты и чрезвычайных ситуаций Михаил Харабаджиу на чрезвычайном заседании правительства в воскресенье. Министр внутренних дел Александр Жиздан в свою очередь сообщил, что в Шолданештском, Резинском и Флорештском районах ливень повалил деревья на дороги, заблокировав проезд. "Ночью около 30 машин, во время стихии оказались заблокированными на трассе. У них разбиты стекла, есть другие повреждения. В настоящее время дороги расчищены, нуждающимся оказана помощь,

In [15]:
symbols = {'BOS': tokenizer.vocab['[unused1]'], 'EOS': tokenizer.vocab['[unused2]'], 
           'PAD': tokenizer.vocab['[PAD]'], 'EOQ': tokenizer.vocab['[unused3]']}

model_path='./model/ru_head/model_step_96000.pt'

In [16]:
summarizer=AbsSummarizer(args,'cuda',get_checkpoint(model_path))

In [17]:
torch.cuda.empty_cache()

In [18]:
translator=build_predictor(args,tokenizer,symbols,summarizer)

In [19]:
import math

In [20]:
def build_comp(data, summarizer, translator, tokenizer, limit=300):
    comp=[]
    for idx in range(len(data[:limit])):
        print(idx)
        text=data[idx]['source']
        target=data[idx]['target'].replace('\xa0', ' ')
        res=list(translator.translate_batch(batch(*[item.to(summarizer.device) for item in prepare_text(text, tokenizer)]), hyp_k=1))
        pred=decode_prediction(tokenizer,res[-1]['predictions'][0][0])
        comp.append((idx,pred,target))
    return comp

In [21]:
comp=build_comp(data, summarizer, translator, tokenizer)

0
torch.Size([1, 194, 768])
torch.Size([10, 194, 768])


C:\Users\Orbis\Downloads\SumPipeline\src\generator.py:346: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:882.)
  finished_hyp = is_finished[i].nonzero().view(-1)


1
torch.Size([1, 382, 768])
torch.Size([10, 382, 768])
2
torch.Size([1, 286, 768])
torch.Size([10, 286, 768])
3
torch.Size([1, 412, 768])
torch.Size([10, 412, 768])
4
torch.Size([1, 179, 768])
torch.Size([10, 179, 768])
5
torch.Size([1, 245, 768])
torch.Size([10, 245, 768])
6
torch.Size([1, 307, 768])
torch.Size([10, 307, 768])
7
torch.Size([1, 252, 768])
torch.Size([10, 252, 768])
8
torch.Size([1, 222, 768])
torch.Size([10, 222, 768])
9
torch.Size([1, 373, 768])
torch.Size([10, 373, 768])
10
torch.Size([1, 248, 768])
torch.Size([10, 248, 768])
11
torch.Size([1, 183, 768])
torch.Size([10, 183, 768])
12
torch.Size([1, 123, 768])
torch.Size([10, 123, 768])
13
torch.Size([1, 84, 768])
torch.Size([10, 84, 768])
14
torch.Size([1, 512, 768])
torch.Size([10, 512, 768])
15
torch.Size([1, 275, 768])
torch.Size([10, 275, 768])
16
torch.Size([1, 211, 768])
torch.Size([10, 211, 768])
17
torch.Size([1, 114, 768])
torch.Size([10, 114, 768])
18
torch.Size([1, 212, 768])
torch.Size([10, 212, 768])
19


147
torch.Size([1, 340, 768])
torch.Size([10, 340, 768])
148
torch.Size([1, 297, 768])
torch.Size([10, 297, 768])
149
torch.Size([1, 384, 768])
torch.Size([10, 384, 768])
150
torch.Size([1, 169, 768])
torch.Size([10, 169, 768])
151
torch.Size([1, 165, 768])
torch.Size([10, 165, 768])
152
torch.Size([1, 322, 768])
torch.Size([10, 322, 768])
153
torch.Size([1, 271, 768])
torch.Size([10, 271, 768])
154
torch.Size([1, 297, 768])
torch.Size([10, 297, 768])
155
torch.Size([1, 455, 768])
torch.Size([10, 455, 768])
156
torch.Size([1, 150, 768])
torch.Size([10, 150, 768])
157
torch.Size([1, 189, 768])
torch.Size([10, 189, 768])
158
torch.Size([1, 114, 768])
torch.Size([10, 114, 768])
159
torch.Size([1, 251, 768])
torch.Size([10, 251, 768])
160
torch.Size([1, 310, 768])
torch.Size([10, 310, 768])
161
torch.Size([1, 187, 768])
torch.Size([10, 187, 768])
162
torch.Size([1, 512, 768])
torch.Size([10, 512, 768])
163
torch.Size([1, 345, 768])
torch.Size([10, 345, 768])
164
torch.Size([1, 313, 768])
t

291
torch.Size([1, 184, 768])
torch.Size([10, 184, 768])
292
torch.Size([1, 154, 768])
torch.Size([10, 154, 768])
293
torch.Size([1, 208, 768])
torch.Size([10, 208, 768])
294
torch.Size([1, 233, 768])
torch.Size([10, 233, 768])
295
torch.Size([1, 199, 768])
torch.Size([10, 199, 768])
296
torch.Size([1, 149, 768])
torch.Size([10, 149, 768])
297
torch.Size([1, 370, 768])
torch.Size([10, 370, 768])
298
torch.Size([1, 87, 768])
torch.Size([10, 87, 768])
299
torch.Size([1, 403, 768])
torch.Size([10, 403, 768])


In [22]:
comp

[(0,
  'Курды передали России тело погибшего возле Пальмиры российского офицера [unused2]',
  'Курды сообщили о передаче России тела геройски погибшего в Сирии офицера'),
 (1,
  'Суд начал рассматривать дело в отношении Олега Шишова о растрате при строительстве океанариума [unused2]',
  'Суд во Владивостоке начал рассмотрение дела о растрате при строительстве океанариума'),
 (2,
  'Всем задержанным по делу о драке на Хованском кладбище предъявлены обвинения в организации убийства [unused2]',
  'СКР предъявил обвинения всем фигурантам дела о драке на Хованском кладбище'),
 (3,
  'Более 50 населенных пунктов Молдавии остались без света из-за непогоды [unused2]',
  'Несколько районов Молдавии серьезно пострадали из-за стихии'),
 (4,
  'Парламенты России , США и Германии обсудили борьбу с ИГ [unused2]',
  'Парламентарии России, Германии и США обсудили Сирию и борьбу с ИГ'),
 (5,
  'Португальская спецслужба выдала россиянина , задержанного в Риме [unused2]',
  'Рим выдал Лиссабону фигуранта

In [18]:
def build_comp_batch(data, translator, tokenizer, batch_size=3, limit=300):
    comp=[]
    for idx in range(0,len(data[:limit]), batch_size):
        print(idx)
        exs=prepare_text_batch([insert_special_toks(d['source']) for d in data[idx:idx+batch_size]], tokenizer)
        target=[d['target'] for d in data[idx:idx+batch_size]]
        res=list(translator.translate_batch(batch(*[item.to(translator.model.device) for item in exs]), hyp_k=1))[0]
        pred=[tokenizer.decode(p[0]) for p in res['predictions']]
        for i, p, t in zip(range(len(pred)), pred, target):
            comp.append((idx+i,p,t))
    return comp

In [18]:
exs=prepare_text_batch([insert_special_toks(d['source']) for d in data[:5]], tokenizer)

In [19]:
for e in exs:
    print(e.shape)

torch.Size([5, 412])
torch.Size([5, 412])
torch.Size([5, 412])


In [20]:
tp=batch(*[item.to(summarizer.device) for item in exs])

In [34]:
[tokenizer.decode(p[0]) for p in tmp[0]['predictions']]

['Курды передали России тело погибшего возле Пальмиры военного [unused2] ИГ [unused2]',
 'Суд начал рассматривать дело в отношении Олега Шишова [unused2]']

In [24]:
idx=14
text=data[idx]['source']
target=data[idx]['target']

res=list(translator.translate_batch(batch(*[item.to(summarizer.device) for item in prepare_text(text, tokenizer)]), hyp_k=1))
print("GOLD: ", target)
print("PREDS: ")
for p, s in zip(res[-1]['predictions'][0],res[-1]['scores'][0]):
    print("Score {:.4f}".format(math.exp(s)), " : ", decode_prediction(tokenizer,p))
print("\nTEXT: ", text)

YESYES
YESYES
GOLD:  Глава Минпромторга рассказал президенту о поддержке транспортного машиностроения
PREDS: 
Score 0.0871  :  Минпромторг рассказал Путину о поддержке отрасли транспортного машиностроения [unused2]

TEXT:  Министр промышленности и торговли Денис Мантуров в ходе рабочей встречи с президентом России Владимиром Путиным проинформировал главу государства о том, какие меры принимаются сейчас для поддержки отдельных отраслей промышленности, в частности транспортного машиностроения. Об этом говорится на сайте Кремля. Глава Минпромторга рассказал, что, в соответствии с решением ограничить эксплуатацию вагонов с продленными сроками службы, Минтранс в конце прошлого года принял нормативные акты, позволяющие в этом году снять с сети и отправить в металлолом около 80 тысяч старых вагонов. Наряду с этим по предложению Минпромторга был увеличен объем финансирования прямых скидок транспортным компаниям. Эта мера должна повысить их заинтересованность в приобретении нового подвижного со

# BERTSUM ORIGINAL

In [47]:
import spacy

In [48]:
nlp = spacy.load('ru2_combined_400ks_96')

In [49]:
nlp.add_pipe(nlp.create_pipe('sentencizer'), first=True)

In [50]:
import pandas as pd

In [51]:
text="Киев внес на рассмотрение Совета Безопасности ООН свой проект резолюции по введению миротворцев в Донбасс."

In [97]:
res=nlp(text)

In [53]:
import additional.extractive_baselines as eb

def get_NE(text, nlp):
    parsed=nlp(text)
    res=[]
    for w in parsed:
        if w.ent_type_:
            if res and w.head.i == res[-1][1]:
                res[-1]=(" ".join([res[-1][0], w.text]),w.i)
            else:
                res.append((w.text, w.i))
    return [i[0] for i in res]

def get_NE(text, nlp):
    parsed=nlp(text)
    return parsed.ents

def NE_overlap(a:str,b:str,nlp,report_token_vise=False):
    a_ne=get_NE(a,nlp)
    b_ne=get_NE(b,nlp)
    if report_token_vise:
        a_ne=sum([i.split() for i in a_ne],[])
        b_ne=sum([i.split() for i in b_ne],[])
    if not a_ne: a_ne=[" "]
    if not b_ne: b_ne=[" "]
    return eb.rouge_n(a_ne,b_ne,return_dict=True)

In [106]:
def get_relations(text, nlp):
    rels=[]
    res=nlp(text)
    for sent in res.sents:
        for w in sent:
            main={}
            for c in w.children:
                if ("mod" in c.dep_ or "pos" in c.dep_) and (w.ent_type or c.ent_type):
                    rels.append((w.lemma_,'is', c.lemma_))
                if "subj" in c.dep_:
                    main['subject']=c.lemma_
                    if 'action' not in main:
                        main['action']=w.lemma_
                if "comp" in c.dep_ and "subj" in main:
                    main['comp']=c.lemma_
                    for cc in c.children:
                        if "obj" in cc.dep_:
                            main['object']=cc.lemma_
                if "obj" in c.dep_:
                    main['object']=c.lemma_
                    if 'action' not in main:
                        main['action']=w.lemma_
            if len(main.keys())>=3:
                if 'comp' in main:
                    main['action']=" ".join(main['action'], main['comp'])
                rels.append((main['subject'], main['action'],main['object']))
    return rels

In [116]:
def get_relations(text, nlp):
    rels=[]
    res=nlp(text)
    for sent in res.sents:
        for w in sent:
            for c in w.children:
                if ("mod" in c.dep_ or "pos" in c.dep_) and (w.ent_type or c.ent_type):
                    rels.append((w.lemma_,'is', c.lemma_))
                if "subj" in c.dep_ or "obj" in c.dep_:
                    rels.append((w.lemma_,'is', c.lemma_))

    return rels

In [107]:
rels=[]
for w in res:
    main={}
    for c in w.children:
        if "mod" in c.dep_:
            rels.append((w.lemma_,'is', c.lemma_))
        if "subj" in c.dep_:
            main['subject']=c.lemma_
            if 'action' not in main:
                main['action']=w.lemma_
        if "obj" in c.dep_:
            main['object']=c.lemma_
            if 'action' not in main:
                main['action']=w.lemma_
    if len(main.keys())==3:
        rels.append((main['subject'], main['action'],main['object']))

In [117]:
get_relations(text, nlp)

[('сила', 'is', 'сша'),
 ('начинают', 'is', 'сила'),
 ('приводить', 'is', 'бомбардировщик'),
 ('генерал', 'is', 'ввс'),
 ('генерал', 'is', 'дэвид'),
 ('ввс', 'is', 'сша'),
 ('рассказать', 'is', 'генерал'),
 ('рассказать', 'is', 'defence'),
 ('отметить', 'is', 'гольфейн'),
 ('привести', 'is', 'бомбардировщик'),
 ('поступать', 'is', 'приказ'),
 ('ведется', 'is', 'подготовка'),
 ('указать', 'is', 'он'),
 ('есть', 'is', 'человек'),
 ('говорят', 'is', 'которые'),
 ('важно', 'is', 'оставаться'),
 ('продумывать', 'is', 'способ'),
 ('мир', 'is', 'это'),
 ('есть', 'is', 'сша'),
 ('сша', 'is', 'только'),
 ('есть', 'is', 'игрок'),
 ('обладать', 'is', 'потенциал'),
 ('добавить', 'is', 'гольфейн'),
 ('стать', 'is', 'генерал'),
 ('называть', 'is', 'страна'),
 ('мочь', 'is', 'сша'),
 ('упоминаются', 'is', 'арсенал'),
 ('арсенал', 'is', 'корея'),
 ('корея', 'is', 'северный'),
 ('конфронтация', 'is', 'пхеньян'),
 ('президент', 'is', 'сша'),
 ('президент', 'is', 'дональд'),
 ('бомбардировщик', 'is', 'bo

In [105]:
for w in res:
    print(w,w.head, w.dep_, list(w.rights), w.ent_type)

Военно начинают advmod [-] 0
- Военно punct [] 0
воздушные силы amod [] 0
силы начинают nsubj [США] 0
США силы nmod [] 385
впервые начинают advmod [года] 0
с года case [] 0
1991 года amod [] 0
года впервые obl [] 0
начинают начинают ROOT [приводить, .] 0
приводить начинают xcomp [бомбардировщики, готовность] 0
свои бомбардировщики det [] 0
ядерные бомбардировщики amod [] 0
бомбардировщики приводить obj [B-52] 0
B-52 бомбардировщики appos [] 0
в готовность case [] 0
боевую готовность amod [] 0
готовность приводить obl [] 0
. начинают punct [] 0
Об этом case [] 0
этом рассказал obl [] 0
генерал рассказал nsubj [ВВС, Дэвид] 0
ВВС генерал nmod [США] 383
США ВВС nmod [] 385
Дэвид генерал appos [Гольфейн] 4317129024397789502
Гольфейн Дэвид flat:name [] 4317129024397789502
рассказал рассказал ROOT [Defence] 0
Defence рассказал iobj [One, .] 383
One Defence flat:foreign [] 383
. Defence punct [] 0
Гольфейн отметил nsubj [] 4317129024397789502
отметил отметил ROOT [поступало, .] 0
, поступало p

In [57]:
get_NE(text,nlp)

(Киев, Совета Безопасности, ООН, Донбасс)

In [12]:
def parse_relations(nlp, text):
    def get_mods(node):
        res=[]
        res.append((node.idx,node.text))
        for ch in node.children:
            if 'mod' in ch.dep_:
                res.append((ch.idx,ch.text))
        return sorted(res)
    tree=nlp(text)
    for it in tree:
        res={"subj":[],
             "action":[],
             "obj":[],
            }
        if 'subj' in it.dep_:
            res['subj']=get_mods(it)
            root=it.head
            res['action']+=get_mods(root)
            for o in root.children:
                if 'obj' in o.dep_:
                    res['obj']+=get_mods(o)
            yield res

In [13]:
next(parse_relations(nlp, text))

{'subj': [(0, 'Киев')],
 'action': [(5, 'внес')],
 'obj': [(55, 'проект'), (62, 'резолюции')]}

In [14]:
def parse(nlp, text):
    pbool = lambda x: '+' if x else '-'
    doc = nlp(text)
    sents=pd.DataFrame(data=[(str(s),) for s in doc.sents], columns=['Sentence']);
    print('NLP={}.{} Text={}'.format(nlp.__class__.__module__, nlp.__class__.__name__, doc))
    display(sents.head())
    toks=pd.DataFrame(data=[(t.shape_, pbool(t.orth_ in nlp.vocab), t.pos_,
                             t.text, t.tag_, t.lemma_, t.dep_, t.head, t.ent_type_) for t in doc],
                     columns=['shape', 'vocab', 'POS', 'text', 'tag', 'lemma', 'dep', 'head', 'Named Entity']);
    display(toks)
    #from spacy import displacy
    #displacy.serve(doc, style='dep')

#text='Трое пострадали в дтп. Несколько человек пострадало в дтп. Один человек пострадал в результате аварии'
text = 'Предвыборный штаб нынешнего президента США Дональда Трампа не назвал безосновательным иск, поданный Демократической партией против него, правительства России и WikiLeaks. Соответствующее заявление штаба опубликовано на его сайте.' 
#text = 'Налоги пропитаны потом всякого, кто трудится .'
parse(nlp, text)

NLP=spacy.lang.ru.Russian Text=Предвыборный штаб нынешнего президента США Дональда Трампа не назвал безосновательным иск, поданный Демократической партией против него, правительства России и WikiLeaks. Соответствующее заявление штаба опубликовано на его сайте.


,Sentence
0,Предвыборный штаб нынешнего президента США Дон...
1,Соответствующее заявление штаба опубликовано н...


,shape,vocab,POS,text,tag,lemma,dep,head,Named Entity
0,Xxxxx,+,ADJ,Предвыборный,ADJ__Case=Nom|Degree=Pos|Gender=Masc|Number=Sing,предвыборный,amod,штаб,
1,xxxx,+,NOUN,штаб,NOUN__Animacy=Inan|Case=Nom|Gender=Masc|Number...,штаб,nsubj,назвал,
2,xxxx,+,ADJ,нынешнего,ADJ__Case=Gen|Degree=Pos|Gender=Masc|Number=Sing,нынешний,amod,президента,
3,xxxx,+,NOUN,президента,NOUN__Animacy=Anim|Case=Gen|Gender=Masc|Number...,президент,nmod,штаб,
4,XXX,+,PROPN,США,PROPN__Animacy=Inan|Case=Gen|Gender=Masc|Numbe...,сша,nmod,президента,LOC
5,Xxxxx,+,PROPN,Дональда,PROPN__Animacy=Anim|Case=Gen|Gender=Masc|Numbe...,дональд,appos,президента,PER
6,Xxxxx,+,PROPN,Трампа,PROPN__Animacy=Anim|Case=Gen|Gender=Masc|Numbe...,трамп,flat:name,Дональда,PER
7,xx,+,PART,не,PART__Polarity=Neg,не,advmod,назвал,
8,xxxx,+,VERB,назвал,VERB__Aspect=Perf|Gender=Masc|Mood=Ind|Number=...,назвать,ROOT,назвал,
9,xxxx,+,ADJ,безосновательным,ADJ__Case=Ins|Degree=Pos|Gender=Masc|Number=Sing,безосновательный,xcomp,назвал,


## Model testing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.pipeline_config import get_default_args, get_checkpoint

In [3]:
args=get_default_args()

In [4]:
args

Namespace(accum_count=20, alpha=0.6, batch_size=140, beam_size=5, bert_data_path='./rubert_data/rusnews', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks='0', label_smoothing=0.1, large=False, load_from_extractive='', log_file='../logs/cnndm.log', lr=1, lr_bert=0.002, lr_dec=0.2, max_grad_norm=0, max_length=150, max_pos=512, max_tgt_len=140, min_length=15, mode='train', model_path='./model/ru_mod', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=50, report_rouge=True, result_path='../results/cnndm', ru='./model/runewsbert', save_checkpoint_steps=2000, seed=666, sep_optim=True, share_emb=False, task='abs', temp_dir='../temp', test_all=False, test_batch_si

In [5]:
args.beam_size=5
args.min_length=5

In [6]:
from transformers import BertTokenizer

C:\Users\Orbis\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Orbis\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Orbis\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Orbis\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passi

In [7]:
tokenizer = BertTokenizer.from_pretrained(args.ru, do_lower_case=False, cache_dir=args.temp_dir)

In [8]:
text="Сборная Бельгии обыграла сборную Англии со счетом 2:0"

In [9]:
" ".join(tokenizer.tokenize(text))

'Сборная Бельгии обыграла сборную Англии со счетом 2 : 0'

In [10]:
from src.bertsum import AbsSummarizer

In [11]:
model_path='./model/ru_mod/model_step_80000_2.pt'
#model_path='./model/ru_mod/model_step_104000.pt'

In [12]:
summarizer=AbsSummarizer(args,'cuda',get_checkpoint(model_path))

In [13]:
text="Представители международной коалиции во главе с США провели встречу с российскими военными по ситуации в Сирии, на которой стороны обсудили меры по предотвращению конфликтов между собой, передает РИА «Новости». По словам представителя коалиции полковника Райана Диллона, «это была встреча лицом к лицу». «Они (участники встречи) представили карты, графики того, где меры по предотвращению конфликтов будут предприниматься, чтобы, во-первых, не стрелять непреднамеренно друг по другу, и, во-вторых, сфокусироваться на борьбе с ИГ», — заявил он. Кроме того, Диллон добавил, что стороны обсудили вопрос продолжения поддержки сил на земле с воздуха в борьбе против террористической группировки «Исламское государство». «Близость разных сил (в Сирии) лишь повышает необходимость обсуждения мер по предотвращению конфликтов», — заключил полковник. Ранее представитель Минобороны России Игорь Конашенков заявил, что сирийские военные дважды подверглись обстрелу из района, где располагаются войска «Сирийских демократических сил» (СДС) при поддержке США. «Исламское государство» — террористическая группировка, деятельность которой запрещена в ряде стран, в том числе в России."

In [14]:
from razdel import sentenize, tokenize

In [15]:
def mark_segments(ids,sep_id):
    seg_id=0
    res=[0]*len(ids)
    for i, tid in enumerate(ids):
        res[i]=seg_id
        if tid==sep_id:
            seg_id=(seg_id+1)%2
    return res

In [16]:
sep_token='[SEP]'
cls_token='[CLS]'

In [17]:
import torch

In [18]:
def prepare_text(text, tokenizer, sep_token='[SEP]', cls_token='[CLS]'):
    tokens=[t for s in sentenize(text) for t in tokenizer.tokenize(cls_token+s.text+sep_token)]
    ids=tokenizer.convert_tokens_to_ids(tokens)
    segs=mark_segments(ids,tokenizer.vocab[sep_token])
    ids=torch.tensor(ids)
    segs=torch.tensor(segs)
    mask=~(ids==0)
    return ids.view(1,-1), segs.view(1,-1), mask.view(1,-1)

In [19]:
prepare_text(text, tokenizer)

(tensor([[   101,  17738,  18192,  22015,   2743,  12604,    869,   4425,  19067,
           25427,    869,  28470,  27706,   1516,  15141,    845,  15882,    128,
            1469,   5785,   7973,  62750,  15875,   1516,  68890,  31969,   5190,
            8023,    128,  11098,   9702,    304,   9580,    326,    132,    102,
             101,   3099,   6335,  16812,  22015,  20843,  70656,  80177,   2838,
             128,    304,   3998,   3370,  23275,  24297,    861,  40368,    326,
             132,    102,    101,    304,   9621,    120,  17145,  15257,    122,
           38296,  21165,    128,  36000,   4105,    128,   4206,  15875,   1516,
           68890,  31969,   8074,  71385,   1523,    128,   5247,    128,   2743,
             130,  10785,    128,   1699,  40499,  91359,   3562,   3487,   1516,
           23685,    128,    851,    128,   2743,    130,  30389,    128,  68387,
           17045,   1469,  13968,    869,  24481,    326,    128,    901,   6309,
            2886

In [21]:
symbols = {'BOS': tokenizer.vocab['[unused1]'], 'EOS': tokenizer.vocab['[unused2]'], 
           'PAD': tokenizer.vocab['[PAD]'], 'EOQ': tokenizer.vocab['[unused3]']}

In [22]:
from src.generator import build_predictor

In [23]:
translator=build_predictor(args,tokenizer,symbols,summarizer)

In [24]:
def format_text(text):
    return text.replace(" ##","").replace('[unused3]','').replace('[unused1]','').replace('[unused2]','').replace(' .',".").replace(' ,',",").replace(' !',"!").replace(' ?',"?")

In [25]:
def decode_prediction(tokenizer,prediction):
    return format_text(" ".join([tokenizer.ids_to_tokens[i] for i in prediction]))

In [26]:
class batch:
    def __init__(self,src,segs,mask):
        self.src=src
        self.segs=segs
        self.mask_src=mask
        self.batch_size=1

In [27]:
text="Военно-воздушные силы США впервые с 1991 года начинают приводить свои ядерные бомбардировщики B-52 в боевую готовность. Об этом генерал ВВС США Дэвид Гольфейн рассказал Defence One. Гольфейн отметил, что официального приказа привести бомбардировщики в 24-часовую боевую готовность пока не поступало, однако подготовка на этом направлении уже ведется. Он указал, что в мире, где «есть люди, которые открыто говорят об использовании ядерного оружия, важно оставаться настороже и продумывать новые способы быть готовым к разным вариантам развития событий». «Это уже не биполярный мир, где есть только США  и СССР. Есть и другие игроки, обладающие ядерным потенциалом», — добавил Гольфейн. При этом генерал не стал называть конкретных стран, с ядерным потенциалом которых могли бы столкнуться США, однако в материале упоминаются «быстро развивающийся ядерный арсенал Северной Кореи и конфронтация президента США Дональда Трампа с Пхеньяном», а также «все более мощные российские вооруженные силы». Boeing B-52 Stratofortress — американский многофункциональный тяжелый сверхдальний межконтинентальный стратегический бомбардировщик-ракетоносец второго поколения, стоящий на вооружении ВВС США с 1955 года. На дозвуковой скорости на высотах до 15 километров он способен нести разные виды оружия, в том числе ядерное. Основная задача, для которой B-52 разрабатывался, — доставить две термоядерные бомбы большой мощности до любой точки СССР."

In [28]:
print(text)

Военно-воздушные силы США впервые с 1991 года начинают приводить свои ядерные бомбардировщики B-52 в боевую готовность. Об этом генерал ВВС США Дэвид Гольфейн рассказал Defence One. Гольфейн отметил, что официального приказа привести бомбардировщики в 24-часовую боевую готовность пока не поступало, однако подготовка на этом направлении уже ведется. Он указал, что в мире, где «есть люди, которые открыто говорят об использовании ядерного оружия, важно оставаться настороже и продумывать новые способы быть готовым к разным вариантам развития событий». «Это уже не биполярный мир, где есть только США  и СССР. Есть и другие игроки, обладающие ядерным потенциалом», — добавил Гольфейн. При этом генерал не стал называть конкретных стран, с ядерным потенциалом которых могли бы столкнуться США, однако в материале упоминаются «быстро развивающийся ядерный арсенал Северной Кореи и конфронтация президента США Дональда Трампа с Пхеньяном», а также «все более мощные российские вооруженные силы». Boeing

In [29]:
import spacy
nlp = spacy.load('ru2_combined_400ks_96')
nlp.add_pipe(nlp.create_pipe('sentencizer'), first=True)

In [30]:
import additional.extractive_baselines as eb

class NE_compare:
    def __init__(self,nlp,token_vise=False):
        self.nlp=nlp
        self.token_vise=token_vise
        
    def process_src(self,src):
        self.src_feats=self.get_NE(src)

    def get_NE(self, text):
        parsed=self.nlp(text)
        return list(set(e.text for e in parsed.ents))

    def NE_overlap(self, a:str, report_token_vise=False):
        a_ne=self.get_NE(a)
        b_ne=self.src_feats

        if report_token_vise:
            a_ne=sum([i.split() for i in a_ne],[])
            b_ne=sum([i.split() for i in b_ne],[])
        if not a_ne: a_ne=[" "]
        if not b_ne: b_ne=[" "]
        return eb.rouge_n(a_ne,b_ne,return_dict=True)
    
    def compare(self, cand):
        score=self.NE_overlap(cand, report_token_vise=self.token_vise)
        return score['Precision']

In [31]:
class NEF_compare:
    def __init__(self,nlp):
        self.nlp=nlp
        
    def process_src(self,src):
        self.src_feats=self.get_relations(src)
    
    def get_relations(self, text):
        rels=[]
        res=self.nlp(text)
        for w in res:
            main={}
            for c in w.children:
                if ("mod" in c.dep_ or "pos" in c.dep_) and (w.ent_type or c.ent_type):
                    rels.append((w.lemma_,'is', c.lemma_))
                if "subj" in c.dep_:
                    main['subject']=c.lemma_
                    if 'action' not in main:
                        main['action']=w.lemma_
                if "obj" in c.dep_:
                    main['object']=c.lemma_
                    if 'action' not in main:
                        main['action']=w.lemma_
            if len(main.keys())==3:
                rels.append((main['subject'], main['action'],main['object']))
        return [" ".join(r) for r in rels]
    
    def get_relations(self, text):
        rels=[]
        res=self.nlp(text)
        rels+=list(set(e.lemma_ for e in res.ents))

        for sent in res.sents:
            for w in sent:
                for c in w.children:
                    if ("mod" in c.dep_ or "pos" in c.dep_) and (w.ent_type or c.ent_type):
                        rels.append((w.lemma_,'is', c.lemma_))
                    if "subj" in c.dep_ or "comp" in c.dep_ or "obj" in c.dep_:
                        rels.append((w.lemma_,'is', c.lemma_))
                    
        return list(set([" ".join(r) for r in rels]))
    
    def compare(self, cand):
        a_ne=self.get_relations(cand)
        b_ne=self.src_feats
        if not a_ne: a_ne=[" "]
        if not b_ne: b_ne=[" "]
        score=eb.rouge_n(a_ne,b_ne,return_dict=True)
        return score['Precision']

In [32]:
def guided_annotation(text, translator, comparer, max_iterations=4):
    raw=batch(*[item.to(translator.model.device) for item in prepare_text(text, tokenizer)])
    comparer.process_src(text)
    final=[]
    old_pos=0
    state=prev_sub=None
    for it in range(max_iterations):
        torch.cuda.empty_cache()
        cooked=list(translator.translate_batch(raw, state=state, prev_sub=prev_sub))
        torch.cuda.empty_cache()
        tmp=decode_prediction(translator.tokenizer,cooked[-1]['predictions'][0][0])
        final.append((tmp, comparer.compare(tmp), cooked[-1]["scores"][0][0]))
        cands=[]
        if not cooked[:-1]:
            break
        print("Iteration:", it)
        for i,c in enumerate(cooked[:-1]):
            sep_count=0
            for w in c[0]:
                if w == 3:
                    sep_count+=1
            if sep_count>it+1:
                break
            tmp=decode_prediction(translator.tokenizer,c[0])[old_pos:]
            cands.append((i, comparer.compare(tmp), c[2]))
            #print(cands[-1])
            print(tmp.strip())
        
        cid=sorted(cands,key=lambda x: (-x[1], -x[2]))[0][0]
        print('Choosen:',cid, '\n')
        
        print('Scores:')
        for item in cands:
            print(*item)
        
        chosen=cooked[cid]
        old_pos=len(decode_prediction(translator.tokenizer,cooked[cid][0]))
        state=chosen[1]
        prev_sub=chosen[0]
    torch.cuda.empty_cache()
    return final

In [33]:
text="В Петропавловске-Камчатском возбуждено уголовное дело по факту смерти 21-летнего местного жителя, к которому не успела бригада скорой медицинской помощи из-за того, что женщина-водитель и ее спутник блокировали машиной путь спецавтомобилю. Об этом сообщается на сайте следственного управления СКР по Камчатскому краю. Дело возбуждено по части 1 статьи 109 УК РФ («Причинение смерти по неосторожности»). По версии следствия, 10 января мужчина вместе со своим братом находился в одной из квартир по улице Циолковского. В какой-то момент ему стало плохо, и брат вызвал скорую помощь. Приехавшая по вызову бригада констатировала смерть молодого человека. Со слов медицинского персонала, своевременно оказать помощь мужчине им помешал автомобиль под управлением женщины, препятствующий проезду скорой. Установлены личности находившихся в легковом автомобиле лиц, препятствующих проезду машины скорой медицинской помощи. Они в ближайшее время будут допрошены следователями, их действиям будет дана правовая оценка. «Кроме этого, будет выясняться, есть ли причинно-следственная связь между смертью мужчины и произошедшим дорожным инцидентом, а также полнота и своевременность действий медицинского персонала», — говорится в сообщении."

In [34]:
print(text)

В Петропавловске-Камчатском возбуждено уголовное дело по факту смерти 21-летнего местного жителя, к которому не успела бригада скорой медицинской помощи из-за того, что женщина-водитель и ее спутник блокировали машиной путь спецавтомобилю. Об этом сообщается на сайте следственного управления СКР по Камчатскому краю. Дело возбуждено по части 1 статьи 109 УК РФ («Причинение смерти по неосторожности»). По версии следствия, 10 января мужчина вместе со своим братом находился в одной из квартир по улице Циолковского. В какой-то момент ему стало плохо, и брат вызвал скорую помощь. Приехавшая по вызову бригада констатировала смерть молодого человека. Со слов медицинского персонала, своевременно оказать помощь мужчине им помешал автомобиль под управлением женщины, препятствующий проезду скорой. Установлены личности находившихся в легковом автомобиле лиц, препятствующих проезду машины скорой медицинской помощи. Они в ближайшее время будут допрошены следователями, их действиям будет дана правовая

In [35]:
guided_annotation(text, translator, NEF_compare(nlp))

C:\Users\Orbis\Downloads\SumPipeline\src\generator.py:344: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:882.)
  for j in got_sent[i].nonzero().view(-1):


Iteration: 0
В Петропавловске-Камчатском следственном управлении СКР возбуждено уголовное дело по факту смерти жителя Петропавловска-Камчатского.
В следственном управлении СКР по Камчатскому краю возбуждено уголовное дело по факту смерти 21-летнего жителя Петропавловска-Камчатского, к которому не успела скорая помощь.
В Петропавловске-Камчатском следственном управлении СКР возбуждено уголовное дело по факту смерти 21-летнего местного жителя, к которому не успела скорая помощь, сообщает « Интерфакс ».
В следственном управлении СКР по Камчатскому краю возбуждено уголовное дело по факту смерти 21-летнего жителя Петропавловска-Камчатского, от которого до смерти не успела скорая помощь.
В следственном управлении СКР по Камчатскому краю возбуждено уголовное дело по факту смерти 21-летнего жителя Петропавловска-Камчатского, от которого до смерти не успела скорая помощь, сообщает пресс-служба управления.
Choosen: 1 

Scores:
0 0.4 -4.1313796043396
1 0.7 -4.221625328063965
2 0.3 -4.025637149810

[('В следственном управлении СКР по Камчатскому краю возбуждено уголовное дело по факту смерти 21-летнего жителя Петропавловска-Камчатского, к которому не успела скорая помощь.  Об этом сообщается на сайте следственного управления. ',
  0.6363636363636364,
  -4.196980953216553),
 ('В следственном управлении СКР по Камчатскому краю возбуждено уголовное дело по факту смерти 21-летнего жителя Петропавловска-Камчатского, к которому не успела скорая помощь.  Об этом сообщается на сайте СКР. ',
  0.5833333333333334,
  -1.447378396987915)]

In [120]:
test=torch.load("../Parser/Crawler/NewsParsing/Testing/bert_data/runews.test.5.bert.pt")

In [38]:
text

'В Петропавловске-Камчатском возбуждено уголовное дело по факту смерти 21-летнего местного жителя, к которому не успела бригада скорой медицинской помощи из-за того, что женщина-водитель и ее спутник блокировали машиной путь спецавтомобилю. Об этом сообщается на сайте следственного управления СКР по Камчатскому краю. Дело возбуждено по части 1 статьи 109 УК РФ («Причинение смерти по неосторожности»). По версии следствия, 10 января мужчина вместе со своим братом находился в одной из квартир по улице Циолковского. В какой-то момент ему стало плохо, и брат вызвал скорую помощь. Приехавшая по вызову бригада констатировала смерть молодого человека. Со слов медицинского персонала, своевременно оказать помощь мужчине им помешал автомобиль под управлением женщины, препятствующий проезду скорой. Установлены личности находившихся в легковом автомобиле лиц, препятствующих проезду машины скорой медицинской помощи. Они в ближайшее время будут допрошены следователями, их действиям будет дана правова

In [39]:
import random

In [130]:
num=random.choice(range(len(test)))
print("RID:",num)
ex=test[num]
text=format_text(" ".join(ex['src_txt'][:8]))
print('TEXT:')
print(text)
guided_annotation(text, translator, NEF_compare(nlp), max_iterations=1)

RID: 1927
TEXT:
За последние 16 лет у россиян пропала уверенность в том, что власти не ущемляют деятельность СМИ. Об этом в понедельник, 6 июня, сообщает РБК со ссылкой на результаты опроса « Левада-центра ». В 2000 году 58 % опрошенных были уверены в отсутствии такой угрозы, однако в 2016 году их число снизилось до 35 %, что является минимальным показателем за все время проведения подобных исследований. При этом 21 % респондентов считают, что « власти ведут наступление на свободу слова и ущемляют независимые СМИ ». Противоположного мнения придерживаются 35 % россиян. Большинство россиян также высказали мнение, что на российских телеканалах есть цензура. В этом уверены 15 % опрошенных, 42 % респондентов склоняются к этому. С этим тезисом не согласны 20 % участников опроса.
Iteration: 0
За последние 16 лет доля россиян, опрошенных « Левада-центром », в 2016 году не составляла 35 %.
За последние 16 лет у россиян пропала уверенность в том, что власти не ущемляют влияние на свободу слова и

[('За последние 16 лет у россиян пропала уверенность в том, что власти не ущемляют работу СМИ, сообщает РБК со ссылкой на результаты опроса « Левада-центра ».  Количество таких заявлений снизилось до 35 %, что является минимальным показателем за всю историю исследования. ',
  0.6,
  -6.552057266235352)]

## Metrics testing

In [42]:
comp=[]
for num,ex in enumerate(test[0:100]):
    print(num)
    src=" ".join(ex['src_txt'][:15])
    gold=ex['tgt_txt'].replace('<q>', ' ')
    nef=NEF_compare(nlp)
    ne=NE_compare(nlp)
    nef.process_src(src)
    ne.process_src(src)
    
    preds=[]
    for i in range(2):
        preds+=guided_annotation(src,translator,NEF_compare(nlp))
    
    scores=[]
    
    for p in preds:
        sc=eb.rouge_n(p[0].split(),gold.split(), return_dict=True)
        scores.append(sc)
    scores[0].update({"NEO":ne.compare(preds[0][0]),"NEF":nef.compare(preds[0][0])})
    tmp=[scores[0]]
    key=1+sorted(zip(scores[1:], range(len(scores[1:]))),key=lambda x: (x[0]['Precision'],x[0]['Recall']))[0][1]
    
    scores[key].update({"NEO":ne.compare(preds[key][0]),"NEF":nef.compare(preds[key][0])})
    tmp.append(scores[key])
    comp.append(tmp)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [43]:
names=["_".join([n,k]) for n, it in zip(['base','neo'], comp[0]) for k in it.keys()]
vals=[sum([list(d.values()) for d in c],[]) for c in comp]

In [51]:
rog=[]
for ex in test[:100]:
    src=" ".join(ex['src_txt'][:3])
    gold=ex['tgt_txt'].replace('<q>', ' ')
    rog.append(eb.rouge_n(src.split(), gold.split(), return_dict=True))

In [52]:
metypes=['Precision','Recall','F1']
pd.DataFrame([[r[n] for n in metypes] for r in rog], columns=metypes).mean()

Precision    0.331262
Recall       0.505405
F1           0.379757
dtype: float64

In [46]:
pd.DataFrame(vals, columns=names).mean()

base_Precision    0.532510
base_Recall       0.419526
base_F1           0.451329
base_NEO          0.638381
base_NEF          0.373586
neo_Precision     0.334643
neo_Recall        0.477390
neo_F1            0.375190
neo_NEO           0.594345
neo_NEF           0.304978
dtype: float64

In [42]:
pd.DataFrame(vals, columns=names).mean()

base_Precision    0.541847
base_Recall       0.429489
base_F1           0.457096
base_NEO          0.634619
base_NEF          0.405766
neo_Precision     0.327829
neo_Recall        0.490428
neo_F1            0.375812
neo_NEO           0.598988
neo_NEF           0.304261
dtype: float64

# BERTSUM (pure Torch)

In [2]:
from transformers import BertTokenizer

C:\Users\Orbis\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Orbis\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Orbis\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Orbis\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passi

In [3]:
model_path='./runewsbert4kk_torch'

In [4]:
import os

In [5]:
tokenizer=BertTokenizer(os.path.join(model_path,'vocab.txt'),do_lower_case=False)

In [6]:
import torch

In [7]:
from transformers import BertModel

In [8]:
import json

def read_dataset(path, encoding="utf-8"):
    res=[]
    with open(path, encoding=encoding) as f:
        for l in f:
            res.append(json.loads(l, encoding=encoding))
    return res

In [9]:
def pad_tensor(vec, pad, dim):
    tmp=[]
    for v in vec:
        pad_size = list(v.shape)
        pad_size[dim] = pad - v.size(dim)
        tmp.append(torch.cat([v, torch.zeros(*pad_size, dtype=v.dtype)], dim=dim))
    return torch.stack(tmp)


class PadCollate:

    def __init__(self, dim=-1):
        self.dim = -1

    def pad_collate(self, batch):
        # find longest sequence
        maxenc_len = max(map(lambda x: x[0].shape[self.dim], batch))
        maxdec_len = max(map(lambda x: x[3].shape[self.dim], batch))
        # pad according to max_len
        return [pad_tensor(b, maxenc_len if i!=3 else maxdec_len, self.dim) for i,b in enumerate(zip(*batch))]

    def __call__(self, batch):
        return self.pad_collate(batch)

In [10]:
class SummaryDataset(torch.utils.data.Dataset):
    def __init__(self, data,limit=None):
        self.data=list(zip(data['input_ids'],data['token_type_ids'],data['attention_mask'],data['labels']))
        if limit:
            self.data=self.data[:limit]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

In [11]:
def make_dataloader(data, randomize=False, batch_size=64,pad_id=0,limit=None):
    dataset = SummaryDataset(data,limit)

    loader = torch.utils.data.DataLoader(dataset, shuffle=randomize, batch_size = batch_size,
                                         collate_fn=PadCollate(pad_id),pin_memory=True)

    return loader

In [12]:
def make_seg_mask(input_ids, token_type_ids, sep_token_id):
    seg_id=0
    for i in range(input_ids.shape[-1]):
        token_type_ids[i]=seg_id
        if input_ids[i] == sep_token_id:
            seg_id=(seg_id+1)%2

In [13]:
from razdel import sentenize

In [17]:
def convert_sentences_to_tensors(tok_text,max_length=512):
    for k in tok_text.keys():
        tok_text[k]=torch.tensor(sum(tok_text[k],[])[:max_length])

In [18]:
def place_bos_eos(tok_sent,bos_id=1,eos_id=2):
    tok_sent[0]=bos_id
    tok_sent[-1]=eos_id
    return torch.tensor(tok_sent)

In [67]:
def process_target_sents(sents,bos_id=1,eos_id=2,sep_id=3):
    res=[]
    for i,s in enumerate(sents):
        tmp=s[1:]
        if i==0:
            tmp=[bos_id]+tmp
        tmp[-1]=sep_id
        if i==len(sents)-1:
            tmp[-1]=eos_id
        res.append(tmp)
    return torch.tensor(sum(res,[]))

In [79]:
class BERTSummaryDataset:
    def __init__(self, tokenizer, data_path:str):
        self.tokenizer = tokenizer
        self.data_path = data_path
        self.data=self.tokenize_data(read_dataset(self.data_path))
        
    def tokenize_data(self,data):
        res={'input_ids':[],
             'token_type_ids':[],
             'attention_mask':[],
             'labels':[]
            }
        for d in data:
            enc=self.tokenize_pair(d['source'],d['target'])
            for k in enc.keys():
                res[k].append(enc[k])
        return res
        
    def tokenize_pair(self, src, tgt):
        src=[s.text for s in sentenize(src)]
        tgt=[s.text for s in sentenize(tgt)]
        enc=self.tokenizer(src)
        convert_sentences_to_tensors(enc)
            
        make_seg_mask(enc['input_ids'],enc['token_type_ids'],self.tokenizer.sep_token_id)
        dec=self.tokenizer(tgt)
        
        enc.update({'labels':process_target_sents(dec['input_ids'])})
        return enc

In [90]:
import os

dataset_dir='./bert_news_data'

In [94]:
train_data=torch.load(os.path.join(dataset_dir,'train.pt'))
val_data=torch.load(os.path.join(dataset_dir,'validation.pt'))

In [95]:
train_data['labels'][0]

tensor([    1, 44186, 10050,  6767, 18116,   128,  1997, 20519,  3590,   851,
        32293, 66586, 22479,   851, 27465, 19456, 28722, 14065, 20715, 59900,
         6123,  4689, 21080, 18089,   132,     3,  4763, 19476, 81078,  8488,
        10686,   612, 11173,  9884, 16323,   132,     2])

In [96]:
train_ld=make_dataloader(train_data,randomize=True,batch_size=32,limit=40)

In [97]:
val_ld=make_dataloader(val_data,randomize=True,batch_size=8,limit=8)

In [98]:
test_data=next(iter(train_ld))

In [99]:
for s,_,_,t in zip(*test_data):
    print(tokenizer.decode(s))
    print(tokenizer.decode(t,clean_up_tokenization_spaces=False))
    

[CLS] По меньшей мере 20 человек погибли и свыше 50 получили ранения в результате схода поезда с рельсов в индийском штате Уттар - Прадеш, сообщает индийский ресурс India Today. [SEP] [CLS] По его информации, пять вагонов железнодорожного состава, следовавшего из Пури в Хардвар, сошли с рельсов вблизи города Музаффарнагар, в 115 километрах от столицы Дели. [SEP] [CLS] Инцидент произошел в 17 : 45 по местному времени ( 15 : 45 по Москве ). [SEP] [CLS] Чиновники министерства железных дорог Индии заявили, что они все еще ожидают подробностей аварии. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [31]:
class PositionalEncoding(torch.nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = torch.nn.Dropout(p=dropout)
        self.dim=d_model
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x * math.sqrt(self.dim)
        #print(x.shape,self.pe.shape)
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [32]:
class TransformerDecoder(torch.nn.Module):
    def __init__(self,embedding,dec_layers=6,d_model=768,nhead=8,dec_dropout=0.2,pos_dropout=0.2,dim_ff=2048):
        super(TransformerDecoder, self).__init__()
        self.decoder_layer = torch.nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead,
                                                         dim_feedforward=dim_ff,dropout=dec_dropout,activation='gelu')
        self.decoder=torch.nn.TransformerDecoder(self.decoder_layer,
                                                 num_layers=dec_layers,norm=torch.nn.LayerNorm(d_model, eps=1e-6))
        self.embedding=embedding
        self.pos_encoder=PositionalEncoding(self.embedding.embedding_dim,dropout=pos_dropout)
        
    def forward(self, tgt, tgt_mask, memory, memory_mask, state=None):
        src=self.embedding(tgt)
        src=self.pos_encoder(src)
        
        #print(src.shape,memory.shape,tgt_mask.shape,memory_mask.shape)
        
        output=self.decoder(tgt=src,memory=memory,tgt_key_padding_mask=tgt_mask, memory_key_padding_mask=memory_mask)
        
        return output, state

In [33]:
bert_path='./runewsbert4kk_torch'

In [34]:
import copy
import math

In [35]:
def make_generator(d_model, vocab_size):
    gen_func = torch.nn.LogSoftmax(dim=-1)
    generator = torch.nn.Sequential(
        torch.nn.Linear(d_model, vocab_size),
        gen_func
    )
    
    return generator

In [50]:
class BERTSUM(torch.nn.Module):
    def __init__(self,model_path=None,bert_path=None,pad_id=0,dec_layers=6,d_model=768,nhead=8,dec_dropout=0.2,dim_ff=2048):
        super(BERTSUM, self).__init__()
        assert model_path or bert_path, "Either model_path or bert_path must be specified"
        
        self.bert=BertModel.from_pretrained(bert_path)
        self.bert.config.gradient_checkpointing=True
        tgt_embeddings = torch.nn.Embedding(self.bert.config.vocab_size, self.bert.config.hidden_size, padding_idx=pad_id)
        #Shared embeddings
        tgt_embeddings.weight = copy.deepcopy(self.bert.embeddings.word_embeddings.weight)  
        
        
        self.decoder=TransformerDecoder(tgt_embeddings, dec_layers=dec_layers,d_model=d_model,nhead=nhead,
                                        dec_dropout=dec_dropout,dim_ff=dim_ff)
        self.padding_token_id=pad_id
        
        self.generator=make_generator(d_model, self.bert.config.vocab_size)
        self.generator[0].weight = self.decoder.embedding.weight
        
    def forward(self,src,seg,attn,tgt, return_logits=False):
        mask_src=attn == 0
        mask_tgt=tgt == self.padding_token_id
        
        last_hidden, _ = self.bert(input_ids=src, attention_mask=attn, token_type_ids=seg,return_dict=False)
        
        tgt=tgt.transpose(0, 1)
        #print(tgt.shape,mask_tgt.shape,last_hidden.shape,mask_src.shape)
        output,state=self.decoder(tgt,mask_tgt,last_hidden,mask_src)
        
        if return_logits:
            logits=self.generator(output)
            return logits,output
        
        return output
    

In [37]:
def get_bert_parameters(model):
    bert_p=[]
    out_p=[]
    for n, p in model.named_parameters():
        if 'embedd' not in n:
            if n.startswith('bert'):
                bert_p.append(p)
            else:
                out_p.append(p)
        else:
            p.requires_grad = False
    return bert_p,out_p

In [75]:
model=BERTSUM(bert_path='./runewsbert4kk_torch')

In [76]:
def get_lr(optimizer):
    return optimizer.param_groups[0]['lr']

In [110]:
import onmt

C:\Users\Orbis\AppData\Roaming\Python\Python36\site-packages\torchaudio\extension\extension.py:14: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
C:\Users\Orbis\AppData\Roaming\Python\Python36\site-packages\torchaudio\backend\utils.py:88: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


In [121]:
tmp_loss=onmt.utils.loss.LabelSmoothingLoss(label_smoothing=0.1, tgt_vocab_size=model.bert.config.vocab_size,ignore_index=0)

In [222]:
class fixed_NMTLoss(onmt.utils.loss.NMTLossCompute):
    def _make_shard_state(self, batch, output, range_, attns=None):
        shard_state = {
            "output": output,
            "target": batch.tgt[: , :, 0],
        }
        return shard_state
    
    def __call__(self,
             batch,
             output,
             attns,
             normalization=1.0,
             shard_size=0,
             trunc_start=0,
             trunc_size=None,
             eval_only=False):

        if trunc_size is None:
            trunc_size = batch.tgt.size(0) - trunc_start
        trunc_range = (trunc_start, trunc_start + trunc_size)
        shard_state = self._make_shard_state(batch, output, trunc_range, attns)
        if shard_size == 0:
            loss, stats = self._compute_loss(batch, **shard_state)
            return loss / float(normalization), stats
        batch_stats = onmt.utils.Statistics()
        for shard in self.shards(shard_state, shard_size,eval_only):
            loss, stats = self._compute_loss(batch, **shard)
            if not eval_only:
                loss.div(float(normalization)).backward()
            batch_stats.update(stats)
        return None, batch_stats
    
    def shards(self,state, shard_size, eval_only=False):
        """
        Args:
            state: A dictionary which corresponds to the output of
                   *LossCompute._make_shard_state(). The values for
                   those keys are Tensor-like or None.
            shard_size: The maximum size of the shards yielded by the model.
            eval_only: If True, only yield the state, nothing else.
                  Otherwise, yield shards.

        Yields:
            Each yielded shard is a dict.

        Side effect:
            After the last shard, this function does back-propagation.
        """

        non_none = dict(onmt.utils.loss.filter_shard_state(state, shard_size))

        keys, values = zip(*((k, [v_chunk for v_chunk in v_split])
                             for k, (_, v_split) in non_none.items()))


        for shard_tensors in zip(*values):
            yield dict(zip(keys, shard_tensors))

        if not eval_only:
            # Assumed backprop'd
            variables = []
            for k, (v, v_split) in non_none.items():
                if isinstance(v, torch.Tensor) and state[k].requires_grad:
                    variables.extend(zip(torch.split(state[k], shard_size),
                                         [v_chunk.grad for v_chunk in v_split]))
            inputs, grads = zip(*variables)
            torch.autograd.backward(inputs, grads)

In [219]:
class tgt_batch:
    def __init__(self,tgt):
        self.tgt=tgt

In [ ]:
def validate(model, validation,loss_fn=None, device='cpu',shd_sz=4):
    with torch.no_grad():
        if not loss_fn:
            #loss_fn=torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
            raise "Loss function not defined"

        val_loss=0
        val_size=0
        model.eval()
        for batch in validation:
            inp,seg,attn,dec=[i.to(device) for i in batch]
            res=model(inp,seg,attn,dec)
            #loss=loss_fn(logits.view(-1, logits.shape[-1]), dec.view(-1))
                        
            tbatch=tgt_batch(dec.view(*dec.shape,1).transpose(0,1))
            stats=loss_fn(tbatch,res,attns=None,shard_size=shd_sz,eval_only=True)[1]
            loss=stats.xent()
            
            val_loss+=res.shape[0] * loss
            val_size+=res.shape[0]
        return val_loss/val_size

def train(model,train,validation=None, device=None,num_epochs=10,lr_enc=1e-3, lr_dec=1e-2,pad_token_id=0, bert_delay=1,
          save_path='./model', restart_optimizer=False, save_interval=10, report_every_steps=0,shd_sz=4):
    if not device:
        device=next(model.parameters()).device
    
    enc_p,dec_p=get_bert_parameters(model)
    optim_enc=torch.optim.Adam(enc_p,lr=lr_enc)
    optim_dec=torch.optim.Adam(dec_p,lr=lr_dec)
    
    scheduler_enc=torch.optim.lr_scheduler.ReduceLROnPlateau(optim_enc)
    scheduler_dec=torch.optim.lr_scheduler.ReduceLROnPlateau(optim_dec)
    
    loss_fn=onmt.utils.loss.LabelSmoothingLoss(label_smoothing=0.1, tgt_vocab_size=model.bert.config.vocab_size,ignore_index=0)
    loss_fn=fixed_NMTLoss(loss_fn,model.generator)
    loss_fn.to(device)
    #loss_fn=torch.nn.CrossEntropyLoss(ignore_index=pad_token_id)
    best_score=None
    start_ep=0

    checkpoint=os.listdir(save_path)
    if checkpoint:
        checkpoint=sorted(checkpoint,key=lambda x: (len(x),x))[-1]
        print("Checkpoint found! Restoring model from {}".format(checkpoint))
        checkpoint=torch.load(os.path.join(save_path,checkpoint))
        if not restart_optimizer:
            optim_enc.load_state_dict(checkpoint['optimizer_state_dict'][0])
            optim_dec.load_state_dict(checkpoint['optimizer_state_dict'][1])
        model.load_state_dict(checkpoint['model_state_dict'])
        start_ep=checkpoint['epoch']+1

    for ep in range(start_ep,num_epochs):
        train_loss=0
        train_size=0
        val_loss=0

        model.train()
        for st, batch in enumerate(train):
            optim_enc.zero_grad()
            optim_dec.zero_grad()
            
            inp,seg,attn,dec=[i.to(device) for i in batch]
            res=model(inp,seg,attn,dec)
            
            #loss=loss_fn(logits.view(-1, logits.shape[-1]), dec.view(-1))
            norm=dec.ne(pad_token_id).sum().item()
            
            tbatch=tgt_batch(dec.view(*dec.shape,1).transpose(0,1))
            stats=loss_fn(tbatch,res,attns=None,shard_size=shd_sz,normalization=norm)[1]
            loss=stats.xent()
            
            train_loss+=res.shape[0] * loss
            train_size+=res.shape[0]

            if ep >= bert_delay:
                optim_enc.step()
            optim_dec.step()

            if report_every_steps>0 and st % report_every_steps==0:
                print("Epoch {} step {} loss: {:.4f} (learning rate: enc {} dec {})".format(ep,st,loss,
                                                                                            get_lr(optim_enc),get_lr(optim_dec)))

            if not validation and st % save_interval==0:
                torch.save({"model_state_dict":model.state_dict(),
                            "optimizer_state_dict":[optim_enc.state_dict(),optim_dec.state_dict()],
                            "epoch":ep},
                           os.path.join(save_path,'model_ep{}_step{}.pt'.format(ep,st)))

        if validation:
            val_loss=validate(model,validation,loss_fn,device,shd_sz)
            if not best_score or val_loss < best_score:
                best_score=val_loss
                torch.save({"model_state_dict":model.state_dict(),
                            "optimizer_state_dict":[optim_enc.state_dict(),optim_dec.state_dict()],
                            "epoch":ep},
                           os.path.join(save_path,'model_ep{}.pt'.format(ep)))
            scheduler_enc.step(val_loss)
            scheduler_dec.step(val_loss)
        print("Epoch {} training xent: {:.4f} (learning rate: enc {} dec {})".format(ep,train_loss/train_size,
                                                                                            get_lr(optim_enc),get_lr(optim_dec)))
        if validation:
            print("Epoch {} validation xent: {:.4f} (learning rate: enc {} dec {})".format(ep,val_loss,
                                                                                            get_lr(optim_enc),get_lr(optim_dec)))

# BERT 2 BERT

In [14]:
from transformers import EncoderDecoderModel,BertTokenizer

In [18]:
import os

In [20]:
import json

def read_dataset(path, encoding="utf-8"):
    res=[]
    with open(path, encoding=encoding) as f:
        for l in f:
            res.append(json.loads(l, encoding=encoding))
    return res

In [113]:
class BERTSummaryDataset:
    def __init__(self, tokenizer, data_path:str):
        self.tokenizer = tokenizer
        self.data_path = data_path
        self.data=self.tokenize_data(read_dataset(self.data_path))
        
    def tokenize_data(self,data):
        raw =[(d['target'],d['source']) for d in data]
        enc=self.tokenizer.batch_encode_plus([d['source'] for d in data],return_tensors='pt', max_length=500,padding='max_length',truncation=True, return_token_type_ids=False)
        dec=self.tokenizer.batch_encode_plus([d['target'] for d in data],return_tensors='pt', max_length=500,padding='max_length',truncation=True, return_token_type_ids=False, return_attention_mask=False)
        enc.update({'labels':dec['input_ids']})
        return enc

In [114]:
train_path="../Parser/Crawler/NewsParsing/Testing/built/train.jsonl"
validation_path="../Parser/Crawler/NewsParsing/Testing/built/validate.jsonl"

In [115]:
model_path='./runewsbert4kk_torch'

In [116]:
tokenizer=BertTokenizer(os.path.join(model_path,'vocab.txt'),do_lower_case=False)

In [117]:
train=BERTSummaryDataset(tokenizer,train_path)

In [119]:
val=BERTSummaryDataset(tokenizer,validation_path)

In [120]:
dataset_dir='./bert_news_data'

In [121]:
import torch

In [123]:
def make_dataloader(data, randomize=False, batch_size=64):
    dataset = torch.utils.data.TensorDataset(data['input_ids'], data['attention_mask'], data['labels']) 

    loader = torch.utils.data.DataLoader(dataset, shuffle=randomize, batch_size = batch_size)

    return loader

In [124]:
train_data=torch.load(os.path.join(dataset_dir,'train.pt'))
val_data=torch.load(os.path.join(dataset_dir,'validation.pt'))

In [125]:
train=make_dataloader(train_data,randomize=True,batch_size=8)

In [126]:
val=make_dataloader(val_data,randomize=False,batch_size=8)

In [169]:
class BERT2BERTSum:
    def __init__(self, model_name,vocab_path,device='cpu'):
        self.device=device
        self.model_name=model_name
        self.model=EncoderDecoderModel.from_pretrained(model_name)
        self.model.to(self.device)
        self.tokenizer=BertTokenizer(vocab_path,do_lower_case=False)
    
    def validate(self,validation,loss_fn=None):
        with torch.no_grad():
            if not loss_fn:
                loss_fn=torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)

            val_loss=0
            val_size=0
            self.model.eval()
            for batch in validation:
                inp,attn,dec=batch
                res=self.model(inp.to(self.device),attn.to(self.device),dec.to(self.device),return_dict=True,use_cache=False)
                loss=loss_fn(res['logits'].view(-1, res['logits'].shape[-1]), dec.view(-1))

                val_loss+=res.shape[0] * loss.item()
                val_size+=res.shape[0]
            return val_loss/val_size
    
    def train(self,train,validation=None,num_epochs=10,lr=1e-3,
              save_path='./model', restart_optimizer=False, save_interval=2000, report_every_steps=0):
        optim=torch.optim.Adam(self.model.parameters(),lr=lr)
        loss_fn=torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
        best_score=None
        start_ep=0
        
        checkpoint=os.listdir(save_path)
        if checkpoint:
            checkpoint=sorted(checkpoint,key=lambda x: (len(x),x))[-1]
            checkpoint=torch.load(os.path.join(save_path,checkpoint))
            if not restart_optimizer:
                optim.load_state_dict(checkpoint['optimizer_state_dict'])
            self.model.load_state_dict(checkpoint['model_state_dict'])
            start_ep=checkpoint['epoch']
            
        for ep in range(start_ep,num_epochs):
            train_loss=0
            train_size=0
            val_loss=0
            
            self.model.train()
            for st, batch in enumerate(train):
                optim.zero_grad()
                inp,attn,dec=batch
                res=self.model(inp.to(self.device),attn.to(self.device),dec.to(self.device),return_dict=True,use_cache=False)
                loss=loss_fn(res['logits'].view(-1, res['logits'].shape[-1]), dec.view(-1))
                
                train_loss+=res['logits'].shape[0] * loss.item()
                train_size+=res['logits'].shape[0]
                
                loss.backward()
                optim.step()
                
                if report_every_steps>0 and st % report_every_steps==0:
                    print("Epoch {} step {} loss: {:.4f}".format(ep,st,loss))
                
                if not validation and st % save_interval==0:
                    torch.save({"model_state_dict":self.model.state_dict(),
                                "optimizer_state_dict":optim.state_dict(),
                                "epoch":ep},
                               os.join(save_path,'model_ep{}_step{}.pt'.format(ep,st)))
            
            if validation:
                val_loss=validate(validation,loss_fn)
                if not best_score or val_loss < best_score:
                    best_score=val_loss
                    torch.save({"model_state_dict":self.model.state_dict(),
                                "optimizer_state_dict":optim.state_dict(),
                                "epoch":ep},
                               os.join(save_path,'model_ep{}.pt'.format(ep)))
                
            print("Epoch {} training xent: {:.4f}".format(ep,train_loss/train_size))
            if validation:
                print("Epoch {} validation xent: {:.4f}".format(ep,val_loss))
            
    
    def inference(self,source):
        self.model.eval()
        data=self.tokenizer.encode(source, return_tensors="pt")
        return self.model(data,self.tokenizer)

In [130]:
summarizer=BERT2BERTSum('./rubert_hugging_face','./runewsbert4kk_torch/vocab.txt')

In [133]:
test_data=next(iter(train))

In [186]:
enc=summarizer.tokenizer.encode("Прибывший в Нью-Йорк на сессию Генассамблеи ООН глава МИД КНДР Ли Ён Хо ответил на недавние угрозы президента США Дональда Трампа в адрес Пхеньяна.",return_tensors='pt')

In [191]:
res=summarizer.model.generate(enc,num_beams=4)

In [193]:
summarizer.tokenizer.decode(res[0])

'[PAD] - - - - - - - - - - - - - - - - - - -'

In [9]:
model=BertModel.from_pretrained('./runewsbert4kk_torch')

In [109]:
model=EncoderDecoderModel.from_pretrained('./rubert_hugging_face')

# mBART

In [1]:
from transformers import MBartForConditionalGeneration, MBartTokenizer

C:\Users\Orbis\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Orbis\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Orbis\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Orbis\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passi

In [2]:
model_name="facebook/mbart-large-cc25"

In [3]:
data_path="../Parser/Crawler/NewsParsing/Testing/built/train.jsonl"

In [4]:
train_path="../Parser/Crawler/NewsParsing/Testing/built/train.jsonl"
validation_path="../Parser/Crawler/NewsParsing/Testing/built/validate.jsonl"

In [5]:
import json

def read_dataset(path, encoding="utf-8"):
    res=[]
    with open(path, encoding=encoding) as f:
        for l in f:
            res.append(json.loads(l, encoding=encoding))
    return res

In [6]:
import os

In [7]:
import torch

In [8]:
class BARTSummaryDataset:
    def __init__(self, tokenizer, data_path:str):
        self.tokenizer = tokenizer
        self.data_path = data_path
        self.data=self.tokenize_data(read_dataset(self.data_path))
        
    def tokenize_data(self,data):
        targets,sources =zip(*[(d['target'],d['source']) for d in data])
        return self.tokenizer.prepare_seq2seq_batch(src_texts=sources,tgt_texts=targets,return_tensors='pt', max_length=600)
    


In [9]:
def make_dataloader(data, randomize=False, batch_size=64):
    dataset = torch.utils.data.TensorDataset(data['input_ids'], data['attention_mask'], data['labels']) 

    loader = torch.utils.data.DataLoader(dataset, shuffle=randomize, batch_size = batch_size)

    return loader

In [10]:
import logging

In [22]:
class MBARTSum:
    def __init__(self, model_name,device='cpu'):
        self.device=device
        self.model_name=model_name
        self.model=MBartForConditionalGeneration.from_pretrained(model_name)
        self.model.to(self.device)
        self.tokenizer=MBartTokenizer.from_pretrained(self.model_name)
    
    def validate(self,validation,loss_fn=None):
        if not loss_fn:
            loss_fn=torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
    
        val_loss=0
        val_size=0
        self.model.eval()
        for batch in validation:
            inp,attn,dec=batch
            res=self.model(inp.to(self.device),attn.to(self.device),dec.to(self.device),return_dict=True,use_cache=False)
            loss=loss_fn(res['logits'].view(-1, res['logits'].shape[-1]), dec.view(-1))

            val_loss+=res.shape[0] * loss.item()
            val_size+=res.shape[0]
        return val_loss/val_size
    
    def train(self,train,validation=None,num_epochs=10,lr=1e-3,
              save_path='./model', restart_optimizer=False, save_interval=2000, report_every_steps=0):
        optim=torch.optim.Adam(self.model.parameters(),lr=lr)
        loss_fn=torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
        best_score=None
        start_ep=0
        
        checkpoint=os.listdir(save_path)
        if checkpoint:
            checkpoint=sorted(checkpoint,key=lambda x: (len(x),x))[-1]
            checkpoint=torch.load(os.path.join(save_path,checkpoint))
            if not restart_optimizer:
                optim.load_state_dict(checkpoint['optimizer_state_dict'])
            self.model.load_state_dict(checkpoint['model_state_dict'])
            start_ep=checkpoint['epoch']
            
        for ep in range(start_ep,num_epochs):
            train_loss=0
            train_size=0
            val_loss=0
            
            self.model.train()
            for st, batch in enumerate(train):
                optim.zero_grad()
                inp,attn,dec=batch
                res=self.model(inp.to(self.device),attn.to(self.device),dec.to(self.device),return_dict=True,use_cache=False)
                loss=loss_fn(res['logits'].view(-1, res['logits'].shape[-1]), dec.view(-1))
                
                train_loss+=res['logits'].shape[0] * loss.item()
                train_size+=res['logits'].shape[0]
                
                loss.backward()
                optim.step()
                
                if report_every_steps>0 and st % report_every_steps==0:
                    print("Epoch {} step {} loss: {:.4f}".format(ep,st,loss))
                
                if not validation and st % save_interval==0:
                    torch.save({"model_state_dict":self.model.state_dict(),
                                "optimizer_state_dict":optim.state_dict(),
                                "epoch":ep},
                               os.join(save_path,'model_ep{}_step{}.pt'.format(ep,st)))
            
            if validation:
                val_loss=validate(validation,loss_fn)
                if not best_score or val_loss < best_score:
                    best_score=val_loss
                    torch.save({"model_state_dict":self.model.state_dict(),
                                "optimizer_state_dict":optim.state_dict(),
                                "epoch":ep},
                               os.join(save_path,'model_ep{}.pt'.format(ep)))
                
            print("Epoch {} training xent: {:.4f}".format(ep,train_loss/train_size))
            if validation:
                print("Epoch {} validation xent: {:.4f}".format(ep,val_loss))
            
    
    def inference(self,source):
        self.model.eval()
        data=self.tokenizer.encode(source, return_tensors="pt")
        return self.model(data)

In [23]:
summarizer=MBARTSum(model_name)

In [24]:
import os

In [26]:
dataset_dir='./news_data'

In [27]:
train_data=torch.load(os.path.join(dataset_dir,'train.pt'))

In [28]:
val_data=torch.load(os.path.join(dataset_dir,'validation.pt'))

In [29]:
train=make_dataloader(train_data,randomize=True,batch_size=8)

In [30]:
val=make_dataloader(val_data,batch_size=8)

In [ ]:
summarizer.train(train,val,report_every_steps=2)

In [10]:
tok = MBartTokenizer.from_pretrained(model_name)

In [30]:
data=read_dataset(data_path)

In [31]:
targets,sources =zip(*[tuple(v for v in d.values()) for d in data])

In [34]:
wtid=tok.get_vocab()
idtw=dict((v,k) for k,v in wtid.items())

In [35]:
text=data[0]['source']

encoded=tok(text, return_tensors="pt")

print(text)
print(" ".join(idtw[int(w)] for ids in encoded['input_ids'] for w in ids))


Госсекретарь США Майк Помпео заявил, что Вашингтон призывает коалицию во главе с Саудовской Аравией прекратить авиаудары в Йемене, передает РИА «Новости». Помпео добавил, что настало время положить конец враждебным действиям. «Соответственно, воздушные удары коалиции должны прекратиться во всех населенных районах Йемена», — сказал он. Ранее сообщалось, что ситуация в Йемене  близка к гуманитарной катастрофе, численность голодающего населения в стране может возрасти до 11,5 млн человек.
▁Гос секретар ь ▁США ▁Май к ▁По м пе о ▁заявил , ▁что ▁Вашингтон ▁приз ывает ▁ко али цию ▁во ▁главе ▁с ▁Сауд овской ▁Ара ви ей ▁прекрати ть ▁авиа у дары ▁в ▁Йе мене , ▁перед ает ▁РИА ▁« Ново сти ». ▁По м пе о ▁добавил , ▁что ▁на стало ▁время ▁по ложить ▁конец ▁в раж де б ным ▁действия м . ▁« Со ответ ственно , ▁ воздушн ые ▁у дары ▁ко али ции ▁должны ▁прекрати ться ▁во ▁всех ▁населен ных ▁районах ▁Йе мена », ▁— ▁сказал ▁он . ▁Ра нее ▁сообщал ось , ▁что ▁ситуация ▁в ▁Йе мене ▁близ ка ▁к ▁гуман и тар ной ▁

In [37]:
train_data=tok.prepare_seq2seq_batch(src_texts=sources[:200],tgt_texts=targets[:200],return_tensors='pt', max_length=400,padding=True)

In [39]:
import torch

In [40]:
#device="cuda" if torch.cuda.is_available() else "cpu"
device='cpu'

In [41]:
summarizer=MBartForConditionalGeneration.from_pretrained(model_name)

In [46]:
data=train_data['input_ids'][:4]
attn=train_data['attention_mask'][:4]
dec=train_data['labels'][:4]

In [59]:
tok.encode("alpha is not beta")

[144, 14612, 83, 959, 51703, 2, 250004]

In [49]:
res=summarizer(data.to(device),attn.to(device),dec,return_dict=True,use_cache=False)

In [53]:
ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=tok.pad_token_id)

In [55]:
loss = ce_loss_fct(res['logits'].view(-1, res['logits'].shape[-1]), dec.view(-1))

In [81]:
loss.backward()

tensor(10.3205, grad_fn=<NllLossBackward>)

In [57]:
res['logits'].view(-1, res['logits'].shape[-1])

tensor([[143.7483,  -4.1864, 159.4612,  ..., 141.8405, 141.6580,  72.0593],
        [132.1032,  -3.8957, 149.5913,  ..., 130.1448, 130.9543,  67.3405],
        [138.9722,  -4.0093, 155.7078,  ..., 139.6829, 140.1628,  71.1721],
        ...,
        [118.6203,  -3.4678, 136.9367,  ..., 119.9778, 121.8618,  63.2117],
        [120.2557,  -3.5137, 138.3320,  ..., 121.3366, 123.1427,  63.8558],
        [120.4639,  -3.5210, 138.4878,  ..., 121.4407, 123.1742,  63.9012]],
       grad_fn=<ViewBackward>)